In [1]:
!pip install sentence_transformers pinecone-client

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.4/179.4 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.4/300.4 kB 10.5 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=8e20da8b08a5f62ad0253c6577492625a0c16ef70e470fb8609f6cea702fec1d
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence_transformers


In [2]:
from sentence_transformers import SentenceTransformer
import pandas as pd
import os, json
import numpy as np
from huggingface_hub import InferenceClient
from tqdm import tqdm
import requests, re
import pinecone
import warnings
from numpy.linalg import norm
warnings.filterwarnings("ignore")

In [3]:
def sentence_embeddings(sentences, model='sentence-transformers/all-MiniLM-L6-v2'):
    # generate embedding vectors for sentences.
    # sentences: list of input sentences,
    # model: sentence-transformers/all-MiniLM-L6-v2 as default
    model = SentenceTransformer(model)
    embeddings = model.encode(sentences,batch_size=128)
    return embeddings

In [37]:
def triplet_similarity(df_questions, df_answers):

    question_embedding_lst = []
    answer_embedding_lst = []

    cur_idx = 0
    cur_embedding_lst = []
    for _, row in df_questions.iterrows():
        idx = int(row[0])
        if (cur_idx + 1) == idx:
            question_embedding_lst.append(np.array(cur_embedding_lst).mean(axis=0))
            cur_idx = idx
            cur_embedding_lst = []
        elif cur_idx != idx:
            question_embedding_lst.append(np.array(cur_embedding_lst).mean(axis=0))
            question_embedding_lst += [np.array([0]) for i in range(idx - cur_idx - 1)]
            cur_idx = idx
            cur_embedding_lst = []
        question = ' '.join([str(row[1]), str(row[2]), str(row[3])])
        cur_embedding_lst.append(sentence_embeddings(question))
    question_embedding_lst.append(np.array(cur_embedding_lst).mean(axis=0))


    cur_idx = 0
    cur_embedding_lst = [np.array([0])] if df_answers["answer_index"].iloc[0] != 0 else []
    for _, row in df_answers.iterrows():
        idx = int(row[0])
        if (cur_idx + 1) == idx:
            answer_embedding_lst.append(np.array(cur_embedding_lst).mean(axis=0))
            cur_idx = idx
            cur_embedding_lst = []
        elif cur_idx != idx:
            answer_embedding_lst.append(np.array(cur_embedding_lst).mean(axis=0))
            answer_embedding_lst += [np.array([0]) for i in range(idx - cur_idx - 1)]
            cur_idx = idx
            cur_embedding_lst = []
        answer = ' '.join([row[1], row[2], row[3]])
        cur_embedding_lst.append(sentence_embeddings(answer))
    answer_embedding_lst.append(np.array(cur_embedding_lst).mean(axis=0))
    return (question_embedding_lst,answer_embedding_lst)

    # print(question_embedding_lst[0])
    # print(answer_embedding_lst[0])

def similarity(cur_idx, question_embedding_lst, answer_embedding_lst):
    similarity = [0] * (cur_idx+1)
    # print(similarity)
    for i in range(cur_idx+1):
        print(i)
        # print(answer_embedding_lst[i])
        # print(question_embedding_lst[i])
        if  answer_embedding_lst[i].all() == np.array([0]).all():
            similarity[i] = 0
        else:
            similarity[i] = np.dot(question_embedding_lst[i], answer_embedding_lst[i])/(norm(question_embedding_lst[i])*norm(answer_embedding_lst[i]))
    similarity_mat = pd.DataFrame()
    similarity_mat['index'] = [idx for idx in range(cur_idx+1)]
    similarity_mat['metric'] = similarity

    return similarity_mat

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
## 1. 7b model

df_questions_0_25 = pd.read_csv('/content/drive/MyDrive/Capstone Project/triplets_output/chatbot_pipeline_output/Llama-2-7b-chat-hf_questions_triples_0_25.csv', header=0)
df_answers_0_25 = pd.read_csv('/content/drive/MyDrive/Capstone Project/triplets_output/chatbot_pipeline_output/Llama-2-7b-chat-hf_answers_triples_0_25.csv', header=0)

df_questions_25_50 = pd.read_csv('/content/drive/MyDrive/Capstone Project/triplets_output/chatbot_pipeline_output/Llama-2-7b-chat-hf_questions_triples_25_50.csv', header=0)
df_answers_25_50 = pd.read_csv('/content/drive/MyDrive/Capstone Project/triplets_output/chatbot_pipeline_output/Llama-2-7b-chat-hf_answers_triples_25_50.csv', header=0)

df_questions_50_75 = pd.read_csv('/content/drive/MyDrive/Capstone Project/triplets_output/chatbot_pipeline_output/Llama-2-7b-chat-hf_questions_triples_50_75.csv', header=0)
df_answers_50_75 = pd.read_csv('/content/drive/MyDrive/Capstone Project/triplets_output/chatbot_pipeline_output/Llama-2-7b-chat-hf_answers_triples_50_75.csv', header=0)

df_questions_75_100 = pd.read_csv('/content/drive/MyDrive/Capstone Project/triplets_output/chatbot_pipeline_output/Llama-2-7b-chat-hf_questions_triples_75_100.csv', header=0)
df_answers_75_100 = pd.read_csv('/content/drive/MyDrive/Capstone Project/triplets_output/chatbot_pipeline_output/Llama-2-7b-chat-hf_answers_triples_75_100.csv', header=0)



In [7]:
# df_questions = pd.concat([df_questions_0_25,df_questions_25_50])
# df_answers = pd.concat([df_answers_0_25,df_answers_25_50])

df_questions = pd.concat([df_questions_0_25,df_questions_25_50,df_questions_50_75,df_questions_75_100])
df_answers = pd.concat([df_answers_0_25,df_answers_25_50,df_answers_50_75,df_answers_75_100])

similarity = triplet_similarity(df_questions, df_answers)

print(similarity)

.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

    index    metric
0       0  0.000000
1       1  0.308973
2       2  0.314459
3       3  0.066085
4       4  0.000000
..    ...       ...
94     94  0.722377
95     95  0.256557
96     96  0.345628
97     97  0.369255
98     98  0.654760

[99 rows x 2 columns]


In [19]:
no_answers_llama2_7b = sum(similarity.metric == 0)
df = similarity[similarity.metric>0]
mean_similarity_llama2_7b = df["metric"].mean(axis=0)

In [18]:
similarity.to_csv (path_or_buf='/content/drive/MyDrive/Capstone Project/triplets_output/evaluation_no_triplet/llama2_7b_model', sep=',', index=True, mode='w', header=True)

In [20]:
## 2. 70b model

df_questions_0_25 = pd.read_csv('/content/drive/MyDrive/Capstone Project/triplets_output/chatbot_pipeline_output/Llama-2-70b-chat-hf_questions_triples_0_25.csv', header=0)
df_answers_0_25 = pd.read_csv('/content/drive/MyDrive/Capstone Project/triplets_output/chatbot_pipeline_output/Llama-2-70b-chat-hf_answers_triples_0_25.csv', header=0)

df_questions_25_50 = pd.read_csv('/content/drive/MyDrive/Capstone Project/triplets_output/chatbot_pipeline_output/Llama-2-70b-chat-hf_questions_triples_25_50.csv', header=0)
df_answers_25_50 = pd.read_csv('/content/drive/MyDrive/Capstone Project/triplets_output/chatbot_pipeline_output/Llama-2-70b-chat-hf_answers_triples_25_50.csv', header=0)

df_questions_50_75 = pd.read_csv('/content/drive/MyDrive/Capstone Project/triplets_output/chatbot_pipeline_output/Llama-2-70b-chat-hf_questions_triples_50_75.csv', header=0)
df_answers_50_75 = pd.read_csv('/content/drive/MyDrive/Capstone Project/triplets_output/chatbot_pipeline_output/Llama-2-70b-chat-hf_answers_triples_50_75.csv', header=0)

df_questions_75_100 = pd.read_csv('/content/drive/MyDrive/Capstone Project/triplets_output/chatbot_pipeline_output/Llama-2-70b-chat-hf_questions_triples_75_100.csv', header=0)
df_answers_75_100 = pd.read_csv('/content/drive/MyDrive/Capstone Project/triplets_output/chatbot_pipeline_output/Llama-2-70b-chat-hf_answers_triples_75_100.csv', header=0)



In [23]:
df_questions = pd.concat([df_questions_0_25,df_questions_25_50,df_questions_50_75,df_questions_75_100])
df_answers = pd.concat([df_answers_0_25,df_answers_25_50,df_answers_50_75,df_answers_75_100])

similarity = triplet_similarity(df_questions, df_answers)

print(similarity)

    index    metric
0       0  0.789140
1       1  0.000000
2       2  0.957736
3       3  0.309448
4       4  0.793266
..    ...       ...
95     95  0.200191
96     96  0.782959
97     97  0.709859
98     98  0.294915
99     99  0.191099

[100 rows x 2 columns]


In [24]:
no_answers_llama2_70b = sum(similarity.metric == 0)
df = similarity[similarity.metric>0]
mean_similarity_llama2_70b = df["metric"].mean(axis=0)

In [25]:
similarity.to_csv (path_or_buf='/content/drive/MyDrive/Capstone Project/triplets_output/evaluation_no_triplet/70b_model', sep=',', index=True, mode='w', header=True)

In [26]:
## 2. bloom model

df_questions_0_25 = pd.read_csv('/content/drive/MyDrive/Capstone Project/triplets_output/chatbot_pipeline_output/bloom_questions_triples_0_25.csv', header=0)
df_answers_0_25 = pd.read_csv('/content/drive/MyDrive/Capstone Project/triplets_output/chatbot_pipeline_output/bloom_answers_triples_0_25.csv', header=0)

df_questions_25_50 = pd.read_csv('/content/drive/MyDrive/Capstone Project/triplets_output/chatbot_pipeline_output/bloom_questions_triples_25_50.csv', header=0)
df_answers_25_50 = pd.read_csv('/content/drive/MyDrive/Capstone Project/triplets_output/chatbot_pipeline_output/bloom_answers_triples_25_50.csv', header=0)

df_questions_50_75 = pd.read_csv('/content/drive/MyDrive/Capstone Project/triplets_output/chatbot_pipeline_output/bloom_questions_triples_50_75.csv', header=0)
df_answers_50_75 = pd.read_csv('/content/drive/MyDrive/Capstone Project/triplets_output/chatbot_pipeline_output/bloom_answers_triples_50_75.csv', header=0)

df_questions_75_100 = pd.read_csv('/content/drive/MyDrive/Capstone Project/triplets_output/chatbot_pipeline_output/bloom_questions_triples_75_100.csv', header=0)
df_answers_75_100 = pd.read_csv('/content/drive/MyDrive/Capstone Project/triplets_output/chatbot_pipeline_output/bloom_answers_triples_75_100.csv', header=0)



In [ ]:
df_questions = pd.concat([df_questions_0_25,df_questions_25_50,df_questions_50_75,df_questions_75_100])
df_answers = pd.concat([df_answers_0_25,df_answers_25_50,df_answers_50_75,df_answers_75_100])

# similarity = triplet_similarity(df_questions, df_answers)
(question_embedding_lst, answer_embedding_lst)=triplet_similarity(df_questions, df_answers)

similarity = similarity(99, question_embedding_lst, answer_embedding_lst)

print(similarity)

In [39]:
question_embedding_lst[66]

array([0])

In [38]:
similarity = similarity(99, question_embedding_lst, answer_embedding_lst)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66


ValueError: ignored

In [ ]:
no_answers_bloom = sum(similarity.metric == 0)
df = similarity[similarity.metric>0]
mean_similarity_bloom = df["metric"].mean(axis=0)

In [ ]:
mean_similarity_bloom

In [ ]:
similarity.to_csv (path_or_buf='/content/drive/MyDrive/Capstone Project/triplets_output/evaluation_no_triplet/bloom_model', sep=',', index=True, mode='w', header=True)

In [ ]:
models = ["llama2-7b","llama2-70b","bloom"]
num_no_answers = [no_answers_llama2_7b ,no_answers_llama2_70b ,no_answers_bloom]
mean_similarity = [mean_similarity_llama2_7b,mean_similarity_llama2_70b,mean_similarity_bloom]
final = {"model": models, "number of no answers":num_no_answers, "similarity":mean_similarity}
final_eval = pd.DataFrame(final)
final_eval


In [ ]:
final_eval.to_csv (path_or_buf='/content/drive/MyDrive/Capstone Project/triplets_output/evaluation_no_triplet/final_comparison', sep=',', index=True, mode='w', header=True)

In [ ]:
similarity_mat = pd.DataFrame()
similarity_mat['index'] = [1,2,3,4]
similarity_mat['metric'] = [[1,2,3,4],[2,3,3,2],[3,2,3,3],[4,1,2,1]]
similarity_mat

In [ ]:
embeddings=[]
for _,embedding in enumerate(similarity_mat["metric"]):
    embeddings.append(embedding)

np.array(embeddings).mean(0)